In [6]:
!pip install Flask opencv-python fpdf2 pyngrok


In [7]:
!ngrok authtoken 2oEi8Fh3Cp4IYkfrnW6DintfSls_7ND39YaKYQxpFM5Q1c4Dw


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [8]:
from flask import Flask, request, jsonify
import cv2
from fpdf import FPDF
from pyngrok import ngrok
import os

app = Flask(__name__)

@app.route('/upload', methods=['POST'])
def upload_image():
    if 'file' not in request.files:
        return jsonify({"error": "No file part"}), 400
    file = request.files['file']
    if file.filename == '':
        return jsonify({"error": "No selected file"}), 400

    image_name = file.filename
    file.save(image_name)  # Save the uploaded file
    print(f"Uploaded image saved as: {image_name}")

    # Read the image using OpenCV
    img = cv2.imread(image_name)
    if img is None:
        return jsonify({"error": "Invalid image file"}), 400

    # Overlay the image name at the bottom of the image
    overlay_text = image_name  # Change this to whatever text you want
    cv2.putText(img, overlay_text, (10, img.shape[0] - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    # Save the modified image
    modified_image_path = f'modified_{image_name}'
    cv2.imwrite(modified_image_path, img)
    print(f"Modified image saved at: {modified_image_path}")

    # Optional: Convert the image to PDF
    pdf_output_path = None
    if request.args.get('generate_pdf', 'false').lower() == 'true':
        pdf = FPDF()
        pdf.add_page()
        pdf.image(modified_image_path, x=10, y=10, w=100)
        pdf_output_path = f'output_{image_name}.pdf'
        pdf.output(pdf_output_path)  # Save the PDF
        print(f"PDF output saved at: {pdf_output_path}")

    # Prepare response
    response = {
        "message": "Image processed successfully",
        "image_link": modified_image_path,
    }
    if pdf_output_path:
        response["pdf_link"] = pdf_output_path

    return jsonify(response), 200

if __name__ == '__main__':
    # Run ngrok to get a public URL
    public_url = ngrok.connect(5000)
    print("Public URL:", public_url)

    # Run the Flask app
    app.run(port=5000)


Public URL: NgrokTunnel: "https://1d7b-34-48-93-175.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [01/Nov/2024 09:07:47] "POST /upload?generate_pdf=true HTTP/1.1" 200 -


Uploaded image saved as: rrr.jpg
Modified image saved at: modified_rrr.jpg
PDF output saved at: output_rrr.jpg.pdf


INFO:werkzeug:127.0.0.1 - - [01/Nov/2024 09:07:50] "POST /upload?generate_pdf=true HTTP/1.1" 200 -


Uploaded image saved as: rrr.jpg
Modified image saved at: modified_rrr.jpg
PDF output saved at: output_rrr.jpg.pdf
